In [15]:
from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI
import pickle 
import re 
from collections import defaultdict
import datetime
import codecs

In [16]:
# This method is done for you. Make sure to put your credentials in twitter.cfg.
def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [17]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)
 

In [18]:
def get_followers():
    followers = []
    f = open ('iit_followers.txt','rb')
    regexp = re.compile(r'p\d+')
    for line in f:
        if regexp.search(str(line)) is not None:
            continue
        else: 
            if line.startswith('aV'):
                line = line[2:]
                followers.append(line.replace("\n",""))
    followers.sort()
    return followers

followers_list = get_followers()

In [ ]:
def print_tweets(outputfile,follower_index):
    tweets = 0
    f = codecs.open(outputfile,'a','utf-8')
    print 'opened file named' , outputfile
    followers = get_followers()[follower_index:]
    for follower in followers:
        request = robust_request(twitter,'statuses/user_timeline',{'screen_name': follower})
        if request == None:
            print "request is None type object"
            continue
        if request.status_code == 401:
            print 'Unauthorized data access. The user does not have access to such information'
            continue
        else:
            if tweets <= 5000:
                f.write(follower + ":\n")
                for tweet in request:
                    if tweet is not None:
                        f.write('\t'+ (tweet['text']) + '\n')
            else:
                f.write("Reached 5000 tweets limit. End of file")
                timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
                newfile = 'outputfile' + timestamp + '.txt'
                print 'calling new file name', newfile
                print_tweets(newfile,followers.index(follower))
        

outputfile = 'outputfile'+ datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+ '.txt'
print_tweets(outputfile,0)
print 'End of collecting tweets of followers'

Got error: {"request":"\/1.1\/statuses\/user_timeline.json","error":"Not authorized."} 
sleeping for 15 minutes.
Got error: {"request":"\/1.1\/statuses\/user_timeline.json","error":"Not authorized."} 
sleeping for 15 minutes.
Got error: {"request":"\/1.1\/statuses\/user_timeline.json","error":"Not authorized."} 
sleeping for 15 minutes.
